In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.insert(1, "Informer-Tensorflow/models")
sys.path.insert(1, "Informer-Tensorflow")
# Load the TensorBoard notebook extension
%load_ext tensorboard
import datetime

In [2]:
from attn import ProbAttention, AttentionLayer

In [3]:
class MultiHeadProbAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        super(MultiHeadProbAttention, self).__init__()
        self.d_model = d_model
        self.num_heads = num_heads
        self.attn = AttentionLayer(ProbAttention(False), self.d_model, self.num_heads)
    def call(self, inputs):
        query = inputs
        key = inputs
        value = inputs
        x = self.attn([query, key, value])
        return x


In [4]:
def positional_encoding(length, depth):
    depth = depth/2

    positions = np.arange(length)[:, np.newaxis]     # (seq, 1)
    depths = np.arange(depth)[np.newaxis, :]/depth   # (1, depth)

    angle_rates = 1 / (10000**depths)         # (1, depth)
    angle_rads = positions * angle_rates      # (pos, depth)

    pos_encoding = np.concatenate(
        [np.sin(angle_rads), np.cos(angle_rads)],
        axis=-1) 

    return tf.cast(pos_encoding, dtype=tf.float32)

class PositionalEmbedding(tf.keras.layers.Layer):
    def __init__(self, d_model):
        super().__init__()
        self.d_model = d_model
        # self.embedding = tf.keras.layers.Embedding(vocab_size, d_model, mask_zero=True) 
        self.pos_encoding = positional_encoding(length=2048, depth=d_model)

    # def compute_mask(self, *args, **kwargs):
    #     return self.embedding.compute_mask(*args, **kwargs)

    def call(self, x):
        length = tf.shape(x)[1]
        # x = self.embedding(x)
        # This factor sets the relative scale of the embedding and positonal_encoding.
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        x = x + self.pos_encoding[tf.newaxis, :length, :]
        return x

In [5]:
class ConvLayer(keras.layers.Layer):
    def __init__(self, c_in):
        super(ConvLayer, self).__init__()
        self.downConv = tf.keras.layers.Conv1D(
                                  filters=c_in,
                                  kernel_size=3,
                                  padding='causal')
        self.norm = tf.keras.layers.BatchNormalization()
        self.activation = tf.keras.layers.ELU()
        self.maxPool = tf.keras.layers.MaxPool1D(pool_size=3)#, strides=2)

    def call(self, x, **kargs):
        x = self.downConv(x)
        x = self.norm(x)
        x = self.activation(x)
        x = self.maxPool(x)
        return x

In [6]:
def models():
    inputs = keras.layers.Input(shape=(2000,1), batch_size=8)
    x = keras.layers.Dense(32, activation="tanh")(inputs)
    x = PositionalEmbedding(32)(x)
    x = MultiHeadProbAttention(64, 32)(x)
    x = ConvLayer(32)(x)
    x = keras.layers.LayerNormalization()(x)
    x = keras.layers.GlobalAveragePooling1D()(x)
    x = keras.layers.Dense(5)(x)
    return keras.Model(inputs, x)

In [7]:
model = models()

[TensorShape([8, 2000, 32]), TensorShape([8, 2000, 32]), TensorShape([8, 2000, 32])]


In [8]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(8, 2000, 1)]            0         
_________________________________________________________________
dense (Dense)                (8, 2000, 32)             64        
_________________________________________________________________
positional_embedding (Positi (8, 2000, 32)             0         
_________________________________________________________________
multi_head_prob_attention (M (8, 2000, 64)             10496     
_________________________________________________________________
conv_layer (ConvLayer)       (8, 666, 32)              6304      
_________________________________________________________________
layer_normalization (LayerNo (8, 666, 32)              64        
_________________________________________________________________
global_average_pooling1d (Gl (8, 32)                   0     

In [9]:
x = tf.ones([8,2000,1])
model(x)

<tf.Tensor: shape=(8, 5), dtype=float32, numpy=
array([[ 1.4916651 , -0.53317744,  0.16782165, -0.01341873,  1.8788977 ],
       [ 1.4916651 , -0.53317744,  0.16782165, -0.01341873,  1.8788977 ],
       [ 1.4916651 , -0.53317744,  0.16782165, -0.01341873,  1.8788977 ],
       [ 1.4916651 , -0.53317744,  0.16782165, -0.01341873,  1.8788977 ],
       [ 1.4916651 , -0.53317744,  0.16782165, -0.01341873,  1.8788977 ],
       [ 1.4916651 , -0.53317744,  0.16782165, -0.01341873,  1.8788977 ],
       [ 1.4916651 , -0.53317744,  0.16782165, -0.01341873,  1.8788977 ],
       [ 1.4916651 , -0.53317744,  0.16782165, -0.01341873,  1.8788977 ]],
      dtype=float32)>

In [10]:
data = np.load("data_shuffled.npz")
X = data["X"]
Y = data["Y"]



In [13]:
model.compile(optimizer="Adam", loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=["accuracy"])
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)


In [12]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

In [14]:
model.fit(x=X[0:904], y=Y[0:904], batch_size=8, epochs=1000, validation_data=(X[904:], Y[904:]),
         callbacks=[tensorboard_callback])

Epoch 1/1000
113/113 [==============================] - 22s 143ms/step - loss: 1.2970 - accuracy: 0.3661 - val_loss: 1.3367 - val_accuracy: 0.3854
Epoch 2/1000
113/113 [==============================] - 13s 117ms/step - loss: 1.1210 - accuracy: 0.4288 - val_loss: 1.6029 - val_accuracy: 0.3021
Epoch 3/1000
113/113 [==============================] - 13s 116ms/step - loss: 1.1246 - accuracy: 0.4344 - val_loss: 1.4732 - val_accuracy: 0.4062
Epoch 4/1000
113/113 [==============================] - 13s 116ms/step - loss: 1.0795 - accuracy: 0.5190 - val_loss: 1.5996 - val_accuracy: 0.3542
Epoch 5/1000
113/113 [==============================] - 13s 117ms/step - loss: 1.0915 - accuracy: 0.4705 - val_loss: 1.2804 - val_accuracy: 0.4062
Epoch 6/1000
113/113 [==============================] - 13s 117ms/step - loss: 1.1073 - accuracy: 0.4972 - val_loss: 2.1961 - val_accuracy: 0.4062
Epoch 7/1000
113/113 [==============================] - 13s 117ms/step - loss: 1.0680 - accuracy: 0.5317 - val_loss: 1

113/113 [==============================] - 13s 116ms/step - loss: 0.9217 - accuracy: 0.5958 - val_loss: 4.1422 - val_accuracy: 0.1979
Epoch 56/1000
113/113 [==============================] - 13s 116ms/step - loss: 0.9591 - accuracy: 0.5619 - val_loss: 5.0040 - val_accuracy: 0.4062
Epoch 57/1000
113/113 [==============================] - 13s 117ms/step - loss: 0.9499 - accuracy: 0.5843 - val_loss: 4.8224 - val_accuracy: 0.1979
Epoch 58/1000
113/113 [==============================] - 13s 117ms/step - loss: 0.9272 - accuracy: 0.5601 - val_loss: 5.0691 - val_accuracy: 0.1979
Epoch 59/1000
113/113 [==============================] - 13s 117ms/step - loss: 0.9585 - accuracy: 0.5697 - val_loss: 7.7074 - val_accuracy: 0.1979
Epoch 60/1000
113/113 [==============================] - 13s 116ms/step - loss: 0.9470 - accuracy: 0.5701 - val_loss: 1.8909 - val_accuracy: 0.3125
Epoch 61/1000
113/113 [==============================] - 13s 117ms/step - loss: 0.9437 - accuracy: 0.5791 - val_loss: 1.7362 -

113/113 [==============================] - 13s 117ms/step - loss: 0.9346 - accuracy: 0.5691 - val_loss: 22.9118 - val_accuracy: 0.2292
Epoch 111/1000
113/113 [==============================] - 13s 116ms/step - loss: 0.9618 - accuracy: 0.5621 - val_loss: 8.5123 - val_accuracy: 0.4062
Epoch 112/1000
113/113 [==============================] - 13s 117ms/step - loss: 1.0143 - accuracy: 0.4853 - val_loss: 6.1650 - val_accuracy: 0.2500
Epoch 113/1000
113/113 [==============================] - 13s 117ms/step - loss: 0.9307 - accuracy: 0.5639 - val_loss: 6.9587 - val_accuracy: 0.1979
Epoch 114/1000
113/113 [==============================] - 13s 116ms/step - loss: 0.9780 - accuracy: 0.5336 - val_loss: 2.5747 - val_accuracy: 0.2292
Epoch 115/1000
113/113 [==============================] - 13s 116ms/step - loss: 0.9760 - accuracy: 0.5493 - val_loss: 4.5700 - val_accuracy: 0.3750
Epoch 116/1000
113/113 [==============================] - 13s 116ms/step - loss: 0.9542 - accuracy: 0.5736 - val_loss: 1

113/113 [==============================] - 13s 116ms/step - loss: 0.8868 - accuracy: 0.6171 - val_loss: 4.5437 - val_accuracy: 0.2812
Epoch 166/1000
113/113 [==============================] - 13s 116ms/step - loss: 0.9134 - accuracy: 0.5730 - val_loss: 2.9158 - val_accuracy: 0.3750
Epoch 167/1000
113/113 [==============================] - 13s 117ms/step - loss: 0.9354 - accuracy: 0.5468 - val_loss: 6.5371 - val_accuracy: 0.3854
Epoch 168/1000
113/113 [==============================] - 13s 116ms/step - loss: 0.9465 - accuracy: 0.6017 - val_loss: 7.2922 - val_accuracy: 0.4062
Epoch 169/1000
113/113 [==============================] - 13s 117ms/step - loss: 0.9309 - accuracy: 0.5828 - val_loss: 4.4407 - val_accuracy: 0.3021
Epoch 170/1000
113/113 [==============================] - 13s 117ms/step - loss: 0.9006 - accuracy: 0.6237 - val_loss: 2.3810 - val_accuracy: 0.3646
Epoch 171/1000
113/113 [==============================] - 13s 116ms/step - loss: 0.9220 - accuracy: 0.6013 - val_loss: 1.

113/113 [==============================] - 13s 117ms/step - loss: 0.8750 - accuracy: 0.6279 - val_loss: 19.4917 - val_accuracy: 0.1979
Epoch 221/1000
113/113 [==============================] - 13s 116ms/step - loss: 0.8794 - accuracy: 0.6063 - val_loss: 3.7327 - val_accuracy: 0.3958
Epoch 222/1000
113/113 [==============================] - 13s 117ms/step - loss: 0.8667 - accuracy: 0.6392 - val_loss: 15.1156 - val_accuracy: 0.1979
Epoch 223/1000
113/113 [==============================] - 13s 116ms/step - loss: 0.8635 - accuracy: 0.6263 - val_loss: 3.3092 - val_accuracy: 0.4167
Epoch 224/1000
113/113 [==============================] - 13s 117ms/step - loss: 0.8600 - accuracy: 0.5850 - val_loss: 3.4400 - val_accuracy: 0.4167
Epoch 225/1000
113/113 [==============================] - 13s 117ms/step - loss: 0.8439 - accuracy: 0.6357 - val_loss: 6.9895 - val_accuracy: 0.2708
Epoch 226/1000
113/113 [==============================] - 13s 117ms/step - loss: 0.8796 - accuracy: 0.6347 - val_loss: 

...

Epoch 770/1000
113/113 [==============================] - 13s 117ms/step - loss: 0.3547 - accuracy: 0.8766 - val_loss: 5.4784 - val_accuracy: 0.3229
Epoch 771/1000
113/113 [==============================] - 13s 116ms/step - loss: 0.3923 - accuracy: 0.8762 - val_loss: 5.0537 - val_accuracy: 0.2917
Epoch 772/1000
113/113 [==============================] - 13s 116ms/step - loss: 0.3575 - accuracy: 0.8788 - val_loss: 6.7993 - val_accuracy: 0.2708
Epoch 773/1000
113/113 [==============================] - 13s 117ms/step - loss: 0.3312 - accuracy: 0.8851 - val_loss: 12.7468 - val_accuracy: 0.2812
Epoch 774/1000
113/113 [==============================] - 13s 116ms/step - loss: 0.3345 - accuracy: 0.8843 - val_loss: 14.5272 - val_accuracy: 0.1979
Epoch 775/1000
113/113 [==============================] - 13s 116ms/step - loss: 0.3800 - accuracy: 0.8555 - val_loss: 28.1927 - val_accuracy: 0.1979
Epoch 776/1000
113/113 [==============================] - 13s 116ms/step - loss: 0.3940 - accuracy: 0.8

Epoch 825/1000
113/113 [==============================] - 13s 117ms/step - loss: 0.3880 - accuracy: 0.8701 - val_loss: 17.4135 - val_accuracy: 0.4062
Epoch 826/1000
113/113 [==============================] - 13s 116ms/step - loss: 0.4341 - accuracy: 0.8618 - val_loss: 3.2313 - val_accuracy: 0.3854
Epoch 827/1000
113/113 [==============================] - 13s 116ms/step - loss: 0.3021 - accuracy: 0.9082 - val_loss: 2.0313 - val_accuracy: 0.5417
Epoch 828/1000
113/113 [==============================] - 13s 116ms/step - loss: 0.3818 - accuracy: 0.8817 - val_loss: 10.3497 - val_accuracy: 0.4062
Epoch 829/1000
113/113 [==============================] - 13s 116ms/step - loss: 0.3341 - accuracy: 0.8924 - val_loss: 10.7676 - val_accuracy: 0.2292
Epoch 830/1000
113/113 [==============================] - 13s 116ms/step - loss: 0.3260 - accuracy: 0.8988 - val_loss: 2.6256 - val_accuracy: 0.4583
Epoch 831/1000
113/113 [==============================] - 13s 117ms/step - loss: 0.3851 - accuracy: 0.8

Epoch 880/1000
113/113 [==============================] - 13s 116ms/step - loss: 0.4099 - accuracy: 0.8620 - val_loss: 13.0008 - val_accuracy: 0.3438
Epoch 881/1000
113/113 [==============================] - 13s 116ms/step - loss: 0.3661 - accuracy: 0.8953 - val_loss: 10.2950 - val_accuracy: 0.1979
Epoch 882/1000
113/113 [==============================] - 13s 117ms/step - loss: 0.3421 - accuracy: 0.8949 - val_loss: 19.2561 - val_accuracy: 0.2396
Epoch 883/1000
113/113 [==============================] - 13s 116ms/step - loss: 0.4014 - accuracy: 0.8816 - val_loss: 5.2603 - val_accuracy: 0.4062
Epoch 884/1000
113/113 [==============================] - 13s 116ms/step - loss: 0.3968 - accuracy: 0.8716 - val_loss: 12.2721 - val_accuracy: 0.1979
Epoch 885/1000
113/113 [==============================] - 13s 116ms/step - loss: 0.3617 - accuracy: 0.8819 - val_loss: 2.6267 - val_accuracy: 0.4688
Epoch 886/1000
113/113 [==============================] - 13s 116ms/step - loss: 0.3512 - accuracy: 0.

Epoch 935/1000
113/113 [==============================] - 13s 117ms/step - loss: 0.3424 - accuracy: 0.8936 - val_loss: 11.0064 - val_accuracy: 0.3021
Epoch 936/1000
113/113 [==============================] - 13s 117ms/step - loss: 0.3228 - accuracy: 0.8960 - val_loss: 5.6880 - val_accuracy: 0.5208
Epoch 937/1000
113/113 [==============================] - 13s 117ms/step - loss: 0.3819 - accuracy: 0.8589 - val_loss: 6.4787 - val_accuracy: 0.2708
Epoch 938/1000
113/113 [==============================] - 13s 117ms/step - loss: 0.3726 - accuracy: 0.8864 - val_loss: 14.8525 - val_accuracy: 0.1979
Epoch 939/1000
113/113 [==============================] - 13s 117ms/step - loss: 0.3061 - accuracy: 0.8969 - val_loss: 6.0619 - val_accuracy: 0.4062
Epoch 940/1000
113/113 [==============================] - 13s 116ms/step - loss: 0.3680 - accuracy: 0.8845 - val_loss: 8.5523 - val_accuracy: 0.3542
Epoch 941/1000
113/113 [==============================] - 13s 115ms/step - loss: 0.3675 - accuracy: 0.86

Epoch 990/1000
113/113 [==============================] - 13s 117ms/step - loss: 0.3564 - accuracy: 0.8678 - val_loss: 2.4221 - val_accuracy: 0.3958
Epoch 991/1000
113/113 [==============================] - 13s 116ms/step - loss: 0.3371 - accuracy: 0.8923 - val_loss: 2.2202 - val_accuracy: 0.5312
Epoch 992/1000
113/113 [==============================] - 13s 117ms/step - loss: 0.3977 - accuracy: 0.8638 - val_loss: 6.7383 - val_accuracy: 0.3542
Epoch 993/1000
113/113 [==============================] - 13s 116ms/step - loss: 0.3573 - accuracy: 0.8860 - val_loss: 10.4776 - val_accuracy: 0.2188
Epoch 994/1000
113/113 [==============================] - 13s 116ms/step - loss: 0.3556 - accuracy: 0.9035 - val_loss: 3.4209 - val_accuracy: 0.4271
Epoch 995/1000
113/113 [==============================] - 13s 117ms/step - loss: 0.3517 - accuracy: 0.8897 - val_loss: 2.1719 - val_accuracy: 0.5104
Epoch 996/1000
113/113 [==============================] - 13s 117ms/step - loss: 0.3390 - accuracy: 0.886